**--- Part One ---**

In [76]:
## Select input data case
## Note: this assumes a plain textfile named `input_{case}` is located in the same folder as this notebook
case = "example" # <- input_example
case = "jfm"
case = "gatton"

# verbose output
verbose = False

import numpy as np
from itertools import cycle, permutations
import re

regex_pattern = r"Valve \b([A-Z]+)\b.*flow rate=(\d+);.*valves? (.*)$"

# Read input
with open(f'input_{case}','r') as f:
    input_lines = f.readlines()

import numpy as np
import multiprocessing as mp
from tqdm.notebook import tqdm
from functools import lru_cache

class Valve():
    def __init__(self, input_string):
        result = re.search(regex_pattern, input_string)
        self.name = result.groups(1)[0]
        self.rate = int(result.groups(1)[1])
        self.distances = {}
        try:
            self.tunnels = [i.strip() for i in result.groups(1)[2].split(',')]
        except AttributeError:
            self.tunnels = [result.groups(1)[2].strip()]
         
class Volcano():
    def __init__(self, total_minutes=30, part=1, show_progress=False):
        self.score = 0
        self.valves = []
        self.valve_dict = {}
        self.total_minutes = total_minutes
        for line in input_lines:
            valve = Valve(line.strip())
            self.valve_dict[valve.name] = valve
            self.valves.append(valve)
        self.minute = 0
        self.total_flow_rate = 0
        self.closed_valves = [valve.name for valve in self.valves]
        self.open_valves = []
        self.potentials = {}
        self.graph = {valve.name: valve.tunnels for valve in self.valves}
        self.target_valves = [valve.name for valve in self.valves if valve.rate>0]
        self.target_valves.sort(key=lambda x: self.valve_dict[x].rate, reverse=True)
        self.distances = {f'{start},{end}':len(self.shortest_path(start,end))-1 for start in self.target_valves for end in self.target_valves}
        for end in self.target_valves:
            self.distances[f'AA,{end}'] = len(self.shortest_path('AA',end))-1
            self.distances[f'{end},AA'] = self.distances[f'AA,{end}']
        if part == 1:
            self.build_paths()
            self.paths = list(self.paths)
            self.scores = [self.score_path(path) for path in self.paths]
            self.best = max(self.scores)
        elif part == 2:
            print('building paths')
            self.build_2_paths(show_progress=show_progress)
            print('finished building paths')
            self.paths = list(self.paths)
            print('scoring paths')
            self.scores = [self.score_2_paths(path) for path in tqdm(self.paths)]
            self.best = max(self.scores)

    def build_paths(self):
        self.paths = set()
        self.grow_path(['AA'],self.target_valves,32)

    def build_2_paths(self, show_progress=False):
        self.paths = set()
        self.grow_2_paths(['AA'],['AA'],self.target_valves,26,26,show_progress=show_progress)
    
    def grow_path(self, path, others, rem):
        for i,other in enumerate(others):
            new_rem = rem - (self.distances[f'{path[-1]},{other}']+1)
            if new_rem <= 0:
                self.paths.add(tuple(path))
                continue
            new_path = path + [other]
            new_others = others[:i] + others[i+1:]
            if len(new_others):
                self.grow_path(new_path, new_others, new_rem)
            else:
                self.paths.add(tuple(new_path))

    def grow_2_paths(self, path1, path2, others, rem1, rem2, show_progress=False):
        if len(others) == 1:
            others = others + ['AA']

        if (self.potential_score(others,rem1) + self.potential_score(others,rem2))>self.score:
            #return (tuple((path1,path2)))

            target_options = list(permutations(others,2))
            if show_progress:
                prog = lambda x: tqdm(x)
                if not isinstance(show_progress, bool):
                    show_progress -= 1
            else:
                prog = lambda x: x
            for target1,target2 in prog(target_options):

                if rem1>0:
                    new_rem1 = rem1 - (self.distances[f'{path1[-1]},{target1}']+1)
                    if new_rem1>0:
                        new_path1 = path1 + [target1]
                    else:
                        new_path1 = path1
                        new_rem1 = rem1
                else:
                    new_path1 = path1
                    new_rem1 = rem1

                if rem2>0:
                    new_rem2 = rem2 - (self.distances[f'{path2[-1]},{target2}']+1)
                    if new_rem2>0:
                        new_path2 = path2 + [target2]
                    else:
                        new_path2 = path2
                        new_rem2 = rem2              
                else:
                    new_path2 = path2
                    new_rem2 = rem2
                
                new_others = [i for i in others if i not in (target1,target2)]

                if (new_rem1<=0 and new_rem2<=0) or len(new_others)==0:
                    self.score = max([self.score_2_paths([new_path1,new_path2]),self.score])
                    self.paths.add((tuple(new_path1),tuple(new_path2)))
                    continue
                else:
                    self.grow_2_paths(new_path1, new_path2, new_others, new_rem1, new_rem2, show_progress=show_progress)


    def potential_score(self, others, remaining):
        n = int(remaining/2)
        rates = [self.valve_dict[i].rate for i in others]
        rates.sort()
        potential = 0
        for i in range(min(n,len(rates))):
            potential += (remaining-i)*rates[i]
        # print(f'{others=}, {remaining=}, {potential=}')
        return potential
        
    def score_target(self, dest, remaining):
        return remaining*self.valve_dict[dest].rate 
    
    def score_path(self, path, verbose=False):
        # print(path)
        score = 0
        remaining = 30
        per_minute_total = 0
        for i,start in enumerate(path[:-1]):
            end = path[i+1]
            dist = len(self.shortest_path(start, end))-1
            remaining -= dist # move to valve
            remaining -= 1 # open valve
            rate = self.valve_dict[end].rate
            per_minute_total += rate
            valve_score = remaining*rate
            score += valve_score
            if verbose:
                print(f"== Minute {30-remaining} ==")
                print(f'moved {dist} times: opened valve {end}: at {rate} per minute for {remaining} minutes for a total of {per_minute_total} per minute')
            if remaining <=1:
                break
        return score

    def score_2_paths(self, paths, remaining=26, verbose=False):
        # print(path)
        score = 0
        per_minute_total = 0
        for path in paths:
            rem = remaining
            for i,start in enumerate(path[:-1]):
                end = path[i+1]
                dist = len(self.shortest_path(start, end))-1
                rem -= dist # move to valve
                rem -= 1 # open valve
                rate = self.valve_dict[end].rate
                per_minute_total += rate
                valve_score = rem*rate
                score += valve_score
                if verbose:
                    print(f"== Minute {30-rem} ==")
                    print(f'moved {dist} times: opened valve {end}: at {rate} per minute for {remaining} minutes for a total of {per_minute_total} per minute')
                if rem <=1:
                    break
        return score


    @lru_cache(maxsize = None)
    def shortest_path(self, start, dest, max_steps=30):
        path = [start]
        if dest in (tunnels := self.valve_dict[path[-1]].tunnels):
            path.append(dest)
            return path
        paths = []
        for tunnel in tunnels:
            if tunnel not in path:
                new_path = path.copy()#.append(tunnel)
                new_path.append(tunnel)
                paths.append(new_path)
        n = 0
        while n<max_steps:
            new_paths = []
            for path in paths:
                if dest in (tunnels := self.valve_dict[path[-1]].tunnels):
                    path.append(dest)
                    return path
                for tunnel in tunnels:
                    if tunnel not in path:
                        new_path = path.copy()
                        new_path.append(tunnel)
                        new_paths.append(new_path)
            paths = new_paths
            n += 1
        return None



In [77]:
volcano = Volcano()
print(max(volcano.scores))

1828


In [73]:
volcano = Volcano(part=2)
volcano.best

building paths
finished building paths
scoring paths


1413

In [48]:
print(volcano.target_valves)
volcano.paths

['HH', 'JJ', 'DD', 'BB', 'EE', 'CC']


[(('AA', 'HH', 'DD', 'JJ'), ('AA', 'EE', 'BB', 'CC')),
 (('AA', 'JJ', 'HH', 'DD'), ('AA', 'CC', 'EE', 'BB')),
 (('AA', 'DD', 'HH', 'EE'), ('AA', 'JJ', 'BB', 'CC')),
 (('AA', 'HH', 'BB', 'EE'), ('AA', 'DD', 'JJ', 'CC')),
 (('AA', 'CC', 'BB', 'DD'), ('AA', 'HH', 'JJ', 'EE')),
 (('AA', 'DD', 'CC', 'BB'), ('AA', 'EE', 'HH', 'JJ')),
 (('AA', 'HH', 'JJ', 'DD'), ('AA', 'EE', 'BB', 'CC')),
 (('AA', 'DD', 'BB', 'JJ'), ('AA', 'EE', 'CC', 'HH')),
 (('AA', 'CC', 'DD', 'BB'), ('AA', 'HH', 'JJ', 'EE')),
 (('AA', 'CC', 'JJ', 'BB'), ('AA', 'EE', 'HH', 'DD')),
 (('AA', 'CC', 'JJ', 'HH'), ('AA', 'DD', 'BB', 'EE')),
 (('AA', 'BB', 'CC', 'HH'), ('AA', 'EE', 'JJ', 'DD')),
 (('AA', 'EE', 'CC', 'HH'), ('AA', 'DD', 'BB', 'JJ')),
 (('AA', 'EE', 'BB', 'CC'), ('AA', 'HH', 'JJ', 'DD')),
 (('AA', 'EE', 'CC', 'BB'), ('AA', 'JJ', 'DD', 'HH')),
 (('AA', 'HH', 'EE', 'CC'), ('AA', 'DD', 'JJ', 'BB')),
 (('AA', 'CC', 'JJ', 'EE'), ('AA', 'HH', 'DD', 'BB')),
 (('AA', 'EE', 'BB', 'DD'), ('AA', 'HH', 'CC', 'JJ')),
 (('AA', '

In [5]:
path = [['AA','JJ','BB','CC'],['AA','DD','FF','EE']]
volcano.score_2_paths(path)

1301

In [86]:
p = ('AA','DD','BB','JJ','HH','EE','CC')
p in volcano.paths

True

In [77]:
volcano.score_path(('AA','DD','BB','JJ','HH','EE','CC'), verbose=True) 

== Minute 2 ==
moved 1 times: opened valve DD: at 20 per minute for 28 minutes for a total of 20 per minute
== Minute 5 ==
moved 2 times: opened valve BB: at 13 per minute for 25 minutes for a total of 33 per minute
== Minute 9 ==
moved 3 times: opened valve JJ: at 21 per minute for 21 minutes for a total of 54 per minute
== Minute 17 ==
moved 7 times: opened valve HH: at 22 per minute for 13 minutes for a total of 76 per minute
== Minute 21 ==
moved 3 times: opened valve EE: at 3 per minute for 9 minutes for a total of 79 per minute
== Minute 24 ==
moved 2 times: opened valve CC: at 2 per minute for 6 minutes for a total of 81 per minute


1651

In [16]:
paths = set()
def grow_2_paths(path1, path2, others, rem1, rem2, show_progress=False):
    if len(others) == 1:
        others = others + ['AA']

    target_options = list(permutations(others,2))
    if show_progress:
        prog = lambda x: tqdm(x)
    else:
        prog = lambda x: x
    for target1,target2 in prog(target_options):

        if rem1>0:
            new_rem1 = rem1 - (volcano.distances[f'{path1[-1]},{target1}']+1)
            if new_rem1>0:
                new_path1 = path1 + [target1]
            else:
                new_path1 = path1
                new_rem1 = rem1                
        else:
            new_path1 = path1
            new_rem1 = rem1

        if rem2>0:
            new_rem2 = rem2 - (volcano.distances[f'{path2[-1]},{target2}']+1)
            if new_rem2>0:
                new_path2 = path2 + [target2]
            else:
                new_path2 = path2
                new_rem2 = rem2                
        else:
            new_path2 = path2
            new_rem2 = rem2
        
        new_others = [i for i in others if i not in (target1,target2)]

        if (new_rem1<=0 and new_rem2<=0) or len(new_others)==0:
            print(f'{path1=}, ')
            paths.add((tuple(new_path1),tuple(new_path2)))
            continue
        else:
            grow_2_paths(new_path1, new_path2, new_others, new_rem1, new_rem2)

grow_2_paths(['AA'],['AA'], volcano.target_valves, 26, 26, show_progress=True)
len(paths)


TypeError: 'set' object is not subscriptable

**--- Part Two ---**

In [102]:
list(permutations(volcano.target_valves,2))

[('BB', 'CC'),
 ('BB', 'DD'),
 ('BB', 'EE'),
 ('BB', 'HH'),
 ('BB', 'JJ'),
 ('CC', 'BB'),
 ('CC', 'DD'),
 ('CC', 'EE'),
 ('CC', 'HH'),
 ('CC', 'JJ'),
 ('DD', 'BB'),
 ('DD', 'CC'),
 ('DD', 'EE'),
 ('DD', 'HH'),
 ('DD', 'JJ'),
 ('EE', 'BB'),
 ('EE', 'CC'),
 ('EE', 'DD'),
 ('EE', 'HH'),
 ('EE', 'JJ'),
 ('HH', 'BB'),
 ('HH', 'CC'),
 ('HH', 'DD'),
 ('HH', 'EE'),
 ('HH', 'JJ'),
 ('JJ', 'BB'),
 ('JJ', 'CC'),
 ('JJ', 'DD'),
 ('JJ', 'EE'),
 ('JJ', 'HH')]